# Prompt Exploration for Image Captioning
We explore different prompts and their impact on image captions. 

## Setup

### Load Libraries

In [65]:
%load_ext jupyter_black

# Libraries
import json
import csv
import pandas as pd
import os
import re
import copy

import requests
from PIL import Image

from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
import torch

from tqdm.notebook import tqdm

# for showing image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

%matplotlib inline

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


### Load Model
For this experiment, all the same model will be used.

In [8]:
# setup pytorch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # for multi-GPU systems, force single GPU
if torch.cuda.is_available():
    device_map = "cuda:0"  # force single, first GPU
    device_type = "cuda"
elif torch.backends.mps.is_available():
    device_map = "auto"
    device_type = "mps"
else:
    device_map = "auto"
    device_type = "cpu"

print(f"Using device: {device_type}")

Using device: cuda


In [9]:
# load model
model_id = "allenai/Molmo-7B-O-0924"
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
)

# print model properties
print("Model ID: ", model_id)
print("Device: ", model.device)
print("Dtype: ", model.dtype)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model ID:  allenai/Molmo-7B-O-0924
Device:  cuda:0
Dtype:  torch.bfloat16


### Helper functions

In [75]:
# captioning function
def generate_caption(
    image_object, model, processor, prompt, temperature=1.0, do_sample=False
):
    """
    Generates a caption for an image.

    Inputs:
    - image_object (pil Image): image to caption.
    - model (torch model): loaded model to use for captioning.
    - processor (torch processor): loaded processor for pre-processing inputs.
    - temperature (float; optional): temperature setting for model, greater than 0. Defaults to 1.0; lower values are more deterministic.
    - do_sample (boolean; optional): whether model should sample probabilities. Defaults to False -- greedy decoding.

    Output:
    - (str): caption for image.
    """
    # process the image and text
    inputs = processor.process(
        images=[image_object],
        text=prompt,
    )

    # move inputs to the correct device and make a batch of size 1
    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

    # generate output; maximum 200 new tokens; stop generation when <|endoftext|> is generated
    output = ""
    with torch.autocast(device_type=device_type, enabled=True, dtype=torch.bfloat16):
        output = model.generate_from_batch(
            inputs,
            GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
            tokenizer=processor.tokenizer,
            use_cache=False,
            temperature=temperature,
            do_sample=do_sample,
        )

        # only get generated tokens; decode them to text
        generated_tokens = output[0, inputs["input_ids"].size(1) :]
        generated_text = processor.tokenizer.decode(
            generated_tokens, skip_special_tokens=True
        )
        output = generated_text.strip()

    return output


def generate_target_dataset(caption_dataset_filename, image_quality_dataset_filename):
    """
    Generates a target dataset for captioning based on VizWiz's image captioning dataset and image quality assessment dataset.

    Inputs:
    - caption_dataset_filename (str): path to caption dataset.
    - image_quality_dataset_filename (str): path to image quality dataset.

    Output:
    - (pd.DataFrame): dataframe containing image annotations and image quality.
    """
    # get images and annotations in one dataframe
    image_annotation_df = None
    with open(caption_dataset_filename) as f:
        # load caption dataset
        caption_dataset_json = json.load(f)

        # combine image files and annotations
        images_df = pd.DataFrame.from_dict(caption_dataset_json["images"])
        annotations_df = pd.DataFrame.from_dict(caption_dataset_json["annotations"])
        grouped_annotations = (
            annotations_df.groupby(["image_id"]).agg(tuple).map(list).reset_index()
        )
        image_annotation_df = images_df.merge(
            grouped_annotations[["image_id", "caption", "is_precanned", "is_rejected"]],
            left_on="id",
            right_on="image_id",
        )

        # vizwiz_url is broken, so fix with https://vizwiz.cs.colorado.edu/*
        image_annotation_df["vizwiz_url"] = image_annotation_df["vizwiz_url"].apply(
            lambda x: x.replace(
                "https://ivc.ischool.utexas.edu/", "https://vizwiz.cs.colorado.edu/"
            )
        )

    # get image quality
    with open(image_quality_dataset_filename) as f:
        # load image quality annotation dataset
        image_quality_dataset_json = json.load(f)
        image_quality_df = pd.DataFrame.from_dict(image_quality_dataset_json)

        # expand object of flaws into individual columns and rename
        image_quality_df = pd.concat(
            [
                image_quality_df.drop(["flaws"], axis=1),
                pd.json_normalize(image_quality_df["flaws"]),
            ],
            axis=1,
        )
        image_quality_df.rename(
            columns={
                "FRM": "framing",
                "BLR": "blur",
                "DRK": "too dark",
                "BRT": "too bright",
                "OBS": "obstruction",
                "OTH": "other",
                "NON": "no issue",
                "ROT": "rotation",
                "caption": "human_captions",
            },
            inplace=True,
        )

    # combine image and quality datasets together
    image_captioning_input = image_annotation_df.merge(
        image_quality_df, left_on="file_name", right_on="image"
    ).drop(["image"], axis=1)

    # remove duplicate id column
    image_captioning_input.drop(["id"], axis=1, inplace=True)

    # reorder columns
    image_captioning_input = image_captioning_input[
        [
            "image_id",
            "file_name",
            "vizwiz_url",
            "text_detected",
            "unrecognizable",
            "framing",
            "blur",
            "obstruction",
            "rotation",
            "too dark",
            "too bright",
            "other",
            "no issue",
            "caption",
            "is_precanned",
            "is_rejected",
        ]
    ]

    # convert image_captioning_input to a list of dictionaries
    image_captioning_input = image_captioning_input.to_dict(orient="records")

    # expand captions, is_precanned, and is_rejected into individual columns
    for index, row in enumerate(image_captioning_input):
        curr_captions = row["caption"]
        curr_precanned = row["is_precanned"]
        curr_rejected = row["is_rejected"]

        # expand captions
        for caption_index in range(0, len(curr_captions)):
            # expand caption
            image_captioning_input[index][f"human_caption_{caption_index + 1}"] = (
                curr_captions[caption_index]
            )

            # expand caption
            image_captioning_input[index][f"is_precanned_{caption_index + 1}"] = (
                curr_precanned[caption_index]
            )

            # expand caption
            image_captioning_input[index][f"is_rejected_{caption_index + 1}"] = (
                curr_rejected[caption_index]
            )

        # remove old rows
        del image_captioning_input[index]["caption"]
        del image_captioning_input[index]["is_precanned"]
        del image_captioning_input[index]["is_rejected"]

    return image_captioning_input


def generate_caption_output(
    image_captioning_input, prompts, image_folder, scratch_path="", use_scratch=False
):
    """
    Generates a caption for an image.

    Inputs:
    - image_captioning_input (pd.DataFrame): dataframe containing image annotations and image quality.
    - prompts (list of tuples): prompts to try. each tuple includes (prompt_name, prompt).
    - image_folder (str): path to image folder.
    - scratch_path (str): path to scratch folder where intermediate files will be stored.
    - use_scratch (bool): whether to save intermediate files

    Output:
    - (list): list of dictionaries containing image annotations and image quality.
    """
    # deepclone input where labels will be
    caption_output = copy.deepcopy(image_captioning_input)

    # create scratch path if it doesn't exist
    if use_scratch:
        os.makedirs(scratch_path, exist_ok=True)

    for index, row in enumerate(tqdm(image_captioning_input)):
        # get image for current annotation
        image_file = os.path.join(image_folder, caption_output[index]["file_name"])
        image = Image.open(image_file)

        # repeat per prompt
        for prompt_name, prompt in prompts:
            # generate caption and store for output
            caption_output[index][f"caption_for_prompt_{prompt_name}"] = (
                generate_caption(image, model, processor, prompt)
            )

        # save scratch file for every 100 images
        if use_scratch:
            if index % 100 == 0:
                with open(
                    os.path.join(scratch_path, f"caption_output_{index}.json"), "w"
                ) as f:
                    json.dump(caption_output, f, indent=4, separators=(",", ": "))

    return caption_output

## Data Loading

In [52]:
dataset_to_caption = generate_target_dataset(
    "./data/caption-dataset/annotations/train.json",
    "./data/image-quality-assessment/annotations/train.json",
)
dataset_to_caption_df = pd.DataFrame.from_dict(dataset_to_caption)
dataset_to_caption_df.head()

,image_id,file_name,vizwiz_url,text_detected,unrecognizable,framing,blur,obstruction,rotation,too dark,...,is_rejected_2,human_caption_3,is_precanned_3,is_rejected_3,human_caption_4,is_precanned_4,is_rejected_4,human_caption_5,is_precanned_5,is_rejected_5
0,0,VizWiz_train_00000000.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,1,3,1,0,0,0,...,False,Quality issues are too severe to recognize vis...,True,True,A bottle of spices in a plastic container layi...,False,False,some basil leaves in a container on a counter,False,False
1,1,VizWiz_train_00000001.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,5,0,0,0,...,False,A kitchen counter the various items on top inc...,False,False,a black tin of Coca Cola placed on a black sur...,False,False,"Black counter with canisters, kettle and can o...",False,False
2,2,VizWiz_train_00000002.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,0,0,0,0,...,False,a can of crushed tomatoes in puree from price ...,False,False,a Price Chopper branded can of crushed tomatoes,False,False,Image is a can of crushed tomatoes in view.,False,False
3,3,VizWiz_train_00000003.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,0,0,0,0,...,False,Screenshot from a smartphone with a case insen...,False,False,image shows a screenshot of a page required ca...,False,False,A screenshot of Spotify page on a cell phone s...,False,False
4,4,VizWiz_train_00000004.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,3,0,0,0,0,...,False,A garden book is sitting on a person's lap.,False,False,a box for a solar garden light laying on someo...,False,False,A blue and yellow box with lights for the gard...,False,False


### Subset which images we want to
There's no need to caption all the images with the set of prompts we want to try. Allow the user to select which images to try using `file_name`.

In [53]:
image_subset = [
    "VizWiz_train_00002685.jpg",
    "VizWiz_train_00004360.jpg",
    "VizWiz_train_00005236.jpg",
    "VizWiz_train_00018277.jpg",
    "VizWiz_train_00008241.jpg",
    "VizWiz_train_00015593.jpg",
    "VizWiz_train_00005191.jpg",
    "VizWiz_train_00002033.jpg",
    "VizWiz_train_00012474.jpg",
    "VizWiz_train_00004571.jpg",
    "VizWiz_train_00020191.jpg",
    "VizWiz_train_00016717.jpg",
    "VizWiz_train_00007812.jpg",
    "VizWiz_train_00000428.jpg",
    "VizWiz_train_00011077.jpg",
    "VizWiz_train_00012338.jpg",
    "VizWiz_train_00012570.jpg",
    "VizWiz_train_00009241.jpg",
    "VizWiz_train_00002353.jpg",
    "VizWiz_train_00000575.jpg",
    "VizWiz_train_00013633.jpg",
    "VizWiz_train_00008078.jpg",
    "VizWiz_train_00000140.jpg",
    "VizWiz_train_00008867.jpg",
    "VizWiz_train_00000078.jpg",
    "VizWiz_train_00000192.jpg",
    "VizWiz_train_00011842.jpg",
    "VizWiz_train_00022167.jpg",
    "VizWiz_train_00003433.jpg",
    "VizWiz_train_00017296.jpg",
    "VizWiz_train_00010915.jpg",
    "VizWiz_train_00000145.jpg",
    "VizWiz_train_00008162.jpg",
    "VizWiz_train_00005952.jpg",
    "VizWiz_train_00000139.jpg",
    "VizWiz_train_00012215.jpg",
    "VizWiz_train_00004920.jpg",
    "VizWiz_train_00010826.jpg",
    "VizWiz_train_00006570.jpg",
    "VizWiz_train_00000070.jpg",
    "VizWiz_train_00009603.jpg",
    "VizWiz_train_00023166.jpg",
    "VizWiz_train_00000621.jpg",
    "VizWiz_train_00010902.jpg",
    "VizWiz_train_00000131.jpg",
    "VizWiz_train_00017219.jpg",
    "VizWiz_train_00015043.jpg",
    "VizWiz_train_00000051.jpg",
    "VizWiz_train_00000149.jpg",
    "VizWiz_train_00012203.jpg",
    "VizWiz_train_00001485.jpg",
    "VizWiz_train_00000142.jpg",
    "VizWiz_train_00000126.jpg",
    "VizWiz_train_00000155.jpg",
    "VizWiz_train_00000558.jpg",
    "VizWiz_train_00000560.jpg",
    "VizWiz_train_00000637.jpg",
    "VizWiz_train_00000506.jpg",
    "VizWiz_train_00000193.jpg",
]

In [56]:
dataset_filtered_df = dataset_to_caption_df[
    dataset_to_caption_df["file_name"].isin(image_subset)
]
dataset_filtered = dataset_filtered_df.to_dict(orient="records")

## Batch Processor
Below, we process all selected data across all prompts. Some constants across runs:
1. `temperature = 1.0` which should balance determinism and randomness.
2. `use_cache = False` to make sure new responses are always generated.
3. `do_sample = False` to use greedy decoding to reduce sampling variance.

In [108]:
prompts = [
    # ("plain", "What's in this image?"),
    # ("plain-succinct", "What is in this image? Please provide a succinct description."),
    # ("blv-baseline", "Describe the image for blind and low-vision users."),
    # (
    #     "original",
    #     """You are a program designed to help blind and low-vision users understand images. Generate an accessible image description that includes key visual and contextual details of the image for blind and low-vision people. Focus on the following principles:
    #     Clarity and Conciseness: Use simple, straightforward language to describe the main subjects and their relationships.;
    #     Relevance: Highlight only essential visual elements that contribute to understanding the image or its purpose.;
    #     Context: Provide contextual information when necessary, such as emotional tone, setting, or action. Avoid assumptions or subjective interpretations.;
    #     Specificity: Include important details like colors, shapes, textures, or text visible in the image, if relevant. Avoid overly general terms or unnecessary details.
    #     Once you generate your caption, shorten it to a succinct, single sentence. Output only the final sentence.
    #     """,
    # ),
    # (
    #     "no-blv-ref",
    #     """You are a program designed to help users understand images. Generate an image description that includes key visual and contextual details of the image. Focus on the following principles:
    #     Clarity and Conciseness: Use simple, straightforward language to describe the main subjects and their relationships.;
    #     Relevance: Highlight only essential visual elements that contribute to understanding the image or its purpose.;
    #     Context: Provide contextual information when necessary, such as emotional tone, setting, or action. Avoid assumptions or subjective interpretations.;
    #     Specificity: Include important details like colors, shapes, textures, or text visible in the image, if relevant. Avoid overly general terms or unnecessary details.
    #     Once you generate your caption, shorten it to a succinct, single sentence. Output only the final sentence.
    #     """,
    # ),
    # (
    #     "abstain",
    #     "You are a helpful assistant for describing images for blind and low-vision individuals. Do not hallucinate with incorrect answers if the image is indescribable. An image is indescribable if the provided image is too blurry, too bright or dark, obstructed, or too ill-framed to recognize correctly. Abstain from providing descriptions if the question is unanswerable.",
    # ),
    (
        "abstain-v2",
        """You are a program designed to help users understand images. Generate an image description that includes key visual and contextual details of the image. Focus on the following principles:
        Clarity and Conciseness: Use simple, straightforward language to describe the main subjects and their relationships.;
        Relevance: Highlight only essential visual elements that contribute to understanding the image or its purpose.;
        Context: Provide contextual information when necessary, such as emotional tone, setting, or action. Avoid assumptions or subjective interpretations.;
        Specificity: Include important details like colors, shapes, textures, or text visible in the image, if relevant. Avoid overly general terms or unnecessary details.
        Abstain: Do not hallucinate with incorrect answers if the image is too blurry, too bright or dark, obstructed, or too ill-framed to recognize correctly. Attempt to generate a caption before abstaining.
        
        Once you generate your caption, shorten it to a succinct, single sentence. Output only the final sentence.
        """,
    ),
]

In [109]:
captions_per_prompt = generate_caption_output(
    dataset_filtered, prompts, "./data/caption-dataset/train"
)

  0%|          | 0/59 [00:00<?, ?it/s]

In [112]:
pd.DataFrame.from_dict(captions_per_prompt)["caption_for_prompt_abstain-v2"].tolist()

['A blurry image shows a hand holding a cylindrical object with a barcode and text, likely in a grocery store setting.',
 'A blurry, dark image shows a blue rectangle with "Frequency Audio" text, likely a screen or display in a dimly lit room.',
 'A blurry image of a soda can on a white surface with a purple wall and brown door in the background.',
 'A blurry image shows a finger partially obscuring a painting with blue, yellow, and white colors.',
 'A blurry image shows a white bag with red text on a wooden surface, possibly a table or floor, with a red object in the background.',
 'A blurry image shows a hand holding a jar of food, with a purple sleeve visible against a dark background.',
 'A blurry image of a cereal box, likely Froot Loops, with a bowl of colorful cereal and milk in the bottom right corner.',
 'A blurry image shows a bottle of medicine on a white surface, with a finger partially obscuring the view.',
 'A blurry image of a Folgers coffee can on a wooden table, with a

### Output to CSV

In [102]:
# create a dataframe
output_df = pd.DataFrame.from_dict(captions_per_prompt)

# save file
outdir = "./data/labeled-data/molmo-model/prompt-testing"
if not os.path.exists(outdir):
    os.mkdir(outdir)

output_df.to_csv(
    f"{outdir}/prompt-testing_03-21-25.csv",
    index=False,
)

# print dataframe
output_df.head()

,image_id,file_name,vizwiz_url,text_detected,unrecognizable,framing,blur,obstruction,rotation,too dark,...,is_rejected_4,human_caption_5,is_precanned_5,is_rejected_5,caption_for_prompt_plain,caption_for_prompt_plain-succinct,caption_for_prompt_blv-baseline,caption_for_prompt_original,caption_for_prompt_no-blv-ref,caption_for_prompt_abstain
0,51,VizWiz_train_00000051.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,2,2,3,0,0,0,...,False,some sort of cream that someone is holding,False,False,The image shows a close-up view of someone hol...,The image shows a close-up of a person's hand ...,The image shows a close-up view of a person's ...,A hand holds a cylindrical package with a barc...,A blurry image shows a hand holding a cylindri...,"I apologize, but I cannot provide a descriptio..."
1,70,VizWiz_train_00000070.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,2,3,5,1,4,1,...,False,Quality issues are too severe to recognize vis...,True,False,The image shows a digital display or sign with...,"The image shows a blurry, dark scene with a bl...",The image shows a dark blue background with a ...,"The image shows a blurry, dark scene with a bl...","The image shows a blurry, dark photograph of a...","I apologize, but I cannot provide a descriptio..."
2,78,VizWiz_train_00000078.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,4,5,0,0,0,...,False,Image contain a partial part of a mountain dew...,False,False,The image shows a close-up of an open can of M...,The image shows a close-up of an open can of M...,The image shows a close-up view of an open can...,"A partially visible soda can with ""Mountain"" t...",A blurry image of a soda can on a white surfac...,"I apologize, but I cannot provide a descriptio..."
3,126,VizWiz_train_00000126.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,False,3,1,4,5,0,0,...,False,A orange on a gray cloth with the photographer...,False,False,The image shows a close-up view of a painting ...,The image shows a close-up view of a person's ...,"The image shows a close-up view of a painting,...",A partially obscured image shows a person's fi...,A partially obscured image shows a person's fi...,"I apologize, but I cannot provide a meaningful..."
4,131,VizWiz_train_00000131.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,1,2,5,0,0,0,...,False,some type of liquid that is in a container,False,False,The image shows a blurry interior scene with a...,The image shows a blurry interior scene with a...,The image shows a blurry interior scene with a...,A blurry image shows a white bag with red text...,A blurry image shows a white bag with red text...,"I apologize, but I cannot provide a descriptio..."


In [104]:
dataset_to_caption_df

,image_id,file_name,vizwiz_url,text_detected,unrecognizable,framing,blur,obstruction,rotation,too dark,...,is_rejected_2,human_caption_3,is_precanned_3,is_rejected_3,human_caption_4,is_precanned_4,is_rejected_4,human_caption_5,is_precanned_5,is_rejected_5
0,0,VizWiz_train_00000000.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,1,3,1,0,0,0,...,False,Quality issues are too severe to recognize vis...,True,True,A bottle of spices in a plastic container layi...,False,False,some basil leaves in a container on a counter,False,False
1,1,VizWiz_train_00000001.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,5,0,0,0,...,False,A kitchen counter the various items on top inc...,False,False,a black tin of Coca Cola placed on a black sur...,False,False,"Black counter with canisters, kettle and can o...",False,False
2,2,VizWiz_train_00000002.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,0,0,0,0,...,False,a can of crushed tomatoes in puree from price ...,False,False,a Price Chopper branded can of crushed tomatoes,False,False,Image is a can of crushed tomatoes in view.,False,False
3,3,VizWiz_train_00000003.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,0,0,0,0,...,False,Screenshot from a smartphone with a case insen...,False,False,image shows a screenshot of a page required ca...,False,False,A screenshot of Spotify page on a cell phone s...,False,False
4,4,VizWiz_train_00000004.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,3,0,0,0,0,...,False,A garden book is sitting on a person's lap.,False,False,a box for a solar garden light laying on someo...,False,False,A blue and yellow box with lights for the gard...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23426,23426,VizWiz_train_00023426.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,1,0,0,0,...,False,A case for the PlayStation two game Major Leag...,False,False,A video game of playing basketball sits on a p...,False,False,"PlayStation 2 game, MLB 2k7, sitting on someon...",False,False
23427,23427,VizWiz_train_00023427.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,False,2,4,5,0,0,0,...,False,A person may be standing behind a bar with bot...,False,False,The portrait section of a bill of paper money.,False,False,Quality issues are too severe to recognize vis...,True,False
23428,23428,VizWiz_train_00023428.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,4,0,0,1,1,...,False,a white box with the date may 02 2012 printed ...,False,False,corner of a white cardboard box with a black b...,False,False,A white box with black label with the date May...,False,False
23429,23429,VizWiz_train_00023429.jpg,https://vizwiz.cs.colorado.edu/VizWiz_visualiz...,True,0,0,0,0,0,0,...,False,A bottle of supplements are laying on a brown ...,False,False,White plastic bottle of healthy hair skin and ...,False,False,White bottle of Healthy Hair Skin and Nails su...,False,False
